### WorldPop: 100m population data
open, explore, attempt to map/associate with Sec2 (lat/lon) sanitation data

In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio

In [2]:
pop_tiff = "data/worldpop/ZMB_ppp_v2c_2015.tif"
pop_tfw = "data/worldpop/ZMB_ppp_v2c_2015.tfw"

In [3]:
zam_aff = (a,b,c,d,e,f) = open(pop_tfw, "r").read().split()

In [4]:
zam_aff

['0.0008333000',
 '0.0000000000',
 '0.0000000000',
 '-0.0008333000',
 '21.9996066369',
 '-8.2243503635']

In [17]:
zam_pop = rasterio.open(pop_tiff)
zam_pop

<open RasterReader name='data/worldpop/ZMB_ppp_v2c_2015.tif' mode='r'>

#### Explore Rasterio obj

In [6]:
zam_pop.name, zam_pop.count, zam_pop.width, zam_pop.height

('data/worldpop/ZMB_ppp_v2c_2015.tif', 1, 14049, 11828)

In [7]:
zam_pop.bounds

BoundingBox(left=21.99918998694486, bottom=-18.080206113491325, right=33.70622168694486, top=-8.223933713491325)

In [15]:
## Affine trans object
zam_pop.affine

Affine(0.0008332999999999998, 0.0, 21.99918998694486,
       0.0, -0.0008333, -8.223933713491325)

In [16]:
## Affine transformation matrix
zam_pop.transform

/Users/dpb/dev/gather/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2910: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


[21.99918998694486,
 0.0008332999999999998,
 0.0,
 -8.223933713491325,
 0.0,
 -0.0008333]

In [25]:
def getxy(robj, i, j):
    ## Tranform row_i, col_j (tiff row/col indexes) to geospatial coordinates
    ## Result is upper-left box corner (https://mapbox.github.io/rasterio/topics/georeferencing.html)
    lon, lat = robj.affine * (i,j)
    return lat, lon

In [78]:
def in_geobox(lat, lon, tl_lat, tl_lon, br_lat, br_lon):
    if lat <= tl_lat and lat >= br_lat and lon >= tl_lon and lon <= br_lon:
        return True
    else:
        return False

In [79]:
getxy(zam_pop, 0, 0), 
getxy(zam_pop, zam_pop.width, zam_pop.height)

(-18.080206113491325, 33.70622168694486)

#### Get cell-wise values, build lat-lon borders

In [11]:
zam_data = zam_pop.read(1)
zam_data.shape

(11828, 14049)

In [39]:
## get index range around lusaka (shrink search space)
kanyama_center_lat = -15.432397
kanyama_center_lon = 28.238683

In [66]:
# Lat indices
lus_row_start, lus_row_end = int(zam_pop.height * 0.71), int(zam_pop.height * 0.74), 

In [69]:
# Lon indices
lus_col_start, lus_col_end = int(zam_pop.width * 0.51), int(zam_pop.width * 0.55)

In [80]:
## Store lat/lon top-left, bottom-right pairs by i,j row-col indedx

grid_latlons = {}

for i in range(zam_pop.height):
    if not(i >= lus_row_start and i <= lus_row_end):
        continue
    for j in range(zam_pop.width):
        if not(j >= lus_col_start and j <= lus_col_end):
            continue
        tl_lat, tl_lon = getxy(zam_pop, i, j)
        br_lat, br_lon = getxy(zam_pop, i + 1, j + 1)
        grid_latlons[(i,j)] = [tl_lat, tl_lon, br_lat, br_lon]

#### Merge w/sec2 data

In [73]:
s2_file = "/Users/dpb/dev/gather/data/tidy_data_questionnaire_section_2.csv"

In [74]:
s2 = pd.read_csv(s2_file)
s2.head(3)

,id,respondent,sex,plot_months,type_of_property,no_of_hhs,no_of_ppl,s2q9a54,type_of_toilet,no_of_toilets,add_toilets,water_source,water_source_other,latitude,longitude,accuracy,altitude
0,id_00001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id_00002,landlord,female,1.0,residential,4.0,25.0,NaN,pit_latrine_lined,1.0,NaN,other_specify,Lusaka water connected tap,NaN,NaN,NaN,NaN
2,id_00003,tenant,male,48.0,residential,4.0,22.0,NaN,NaN,NaN,yes_4,individual_connection,NaN,NaN,NaN,NaN,NaN


In [ ]:
pop_idx_col = []
pop_val_vol = []
found = False

for i, row in s2.iterrows():
    for tif_idx, tif_box in grid_latlons.items():
        if in_geobox(row.latitude, row.longitude, *tif_box):
            pop_idx_col.append(tif_idx)
            pop_val_vol.append(zam_data[tif_idx[0], tif_idx[1]])
            found = True
            break
    if found:
        found = False
        continue
            